# Linear Regression



Starting off with usual imports, stuff we have seen before already

In [110]:
#Dataframe and array manipulation
import pandas as pd
import numpy as np

#For visualization
import plotly
import plotly.express as px

## Importing the Income Data

To start, let's import the income data that we were looking at earlier. For simplicity, let's *only* look at the data ranging from $15k to $70k.

Let's start with creating a dataframe to visualize the data.

In [111]:
# Importing the data into a pd dataframe
URL = "https://raw.githubusercontent.com/ishaandey/node/master/week-8/workshop/lin-reg/income.csv"
data = pd.read_csv(URL)
data.head()

,Unnamed: 0,income,happiness
0,1,3.862647,2.314489
1,2,4.979381,3.433490
2,3,4.923957,4.599373
3,4,3.214372,2.791114
4,5,7.196409,5.596398


Let's go ahead and drop the unnecessary column and multiply the income by 10000 to match dollars. (Throwback to data cleaning/manipulation)

In [112]:
# Dropping the first column
data = data.drop(columns=["Unnamed: 0"])
data['income'] = data['income']*10000
data.head()

,income,happiness
0,38626.474184,2.314489
1,49793.813825,3.433490
2,49239.569362,4.599373
3,32143.724388,2.791114
4,71964.092511,5.596398


That's much better.

In [113]:
data.shape

(498, 2)

## Let's visualize the data

In [114]:
# Simple scatter plot
px.scatter(data, x='income', y='happiness',
    labels = {"income" : "Income (in Euros)",
              'happiness' : 'Happiness Score (0 to 10)'
              }
)

Linear regression works very well with data that has a correlation with each other. Since both of the columns are already in numerical form, we don't have to do much in terms of cleaning/modifying the data.

Let's get it ready for the model now.

## Implementing Linear Regression

In [115]:
# Split into X and y and do train_test_split (this should be familiar)
from sklearn.model_selection import train_test_split

X = data.drop(columns=['happiness'])
y = data.happiness

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [116]:
# Fit a LinearRegression to the data
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [117]:
# Predict on the testing data and compare it with the actual data
predicted = reg.predict(X_test)
actual = np.array(y_test)

print('Look at first 5 predictions:')
print('Predicted: ',predicted[:5].round(2))
print('Actual:    ',actual[:5].round(2))

Look at first 5 predictions:
Predicted:  [3.22 2.22 3.2  1.79 1.74]
Actual:     [4.75 4.16 2.3  2.31 2.86]


As you can see from the prediction/actual, none of these are exactly correct. It's kind of unrealistic to expect the model to accurately predict a value exactly. Let's check out what the model looks like through a scatter plot.

$$ y = \beta_{1} x + \beta_{0} $$

There's actually a way to get the coefficients that the model creates.

In [118]:
# Get the coefficients and y-intercept
coef = reg.coef_
intercept = reg.intercept_
print("beta_1 = ", coef)
print("beta_0 = ", intercept)

# Find the first and second point
# The first point will just be (0, intercept)
x_0 = 15000
y_0 = coef[0]*x_0 + intercept
x_1 = 75000
y_1 = coef[0]*x_1 + intercept
print("Point 1: [", x_0, ",", y_0, "]")
print("Point 2: [", x_1, ",", y_1, "]")

beta_1 =  [7.24768702e-05]
beta_0 =  0.14170387223712044
Point 1: [ 15000 , 1.228856924664379 ]
Point 2: [ 75000 , 5.5774691343734135 ]


In [119]:
# Graphs the data and the line on plotly
fig = px.scatter(data, x="income", y="happiness")
fig.add_shape(type='line', xref="x", yref="y",
    x0 = x_0, y0 = y_0, x1 = x_1, y1 = y_1,
    line = dict(
        color = "red",
        width = 4,
    )   
)
fig.show()

## Metrics

You can't really look for the accuracy of a regression model like you would for classification models. A common way to look at how good a regression model is, is through the **Mean Squared Error**.

$$  \frac{1}{n}\Sigma_{i=1}^{n}{\Big(y_a -y_p\Big)^2} $$

In [120]:
# Get the mean squared error of the linear regression model
from sklearn.metrics import mean_squared_error

predicted = reg.predict(X_test)
actual = np.array(y_test)

mse = mean_squared_error(predicted, actual)
print(mse.round(4))

0.5901


# Trying it out on complex data

Let's check out a different dataset. This one looks at the different medical charges a patient got from their visit to the hospital.

In [121]:
# Importing data into a dataframe
URL = "https://raw.githubusercontent.com/ishaandey/node/master/week-8/workshop/lin-reg/med_charges.csv"
med = pd.read_csv(URL)

# Clean the dataset
med = med.drop("Unnamed: 0", axis=1)
med.head()

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520


Let's try to predict the medical charge someone would have with certain characteristics (age, bmi, etc.)

In [122]:
# Correlation matrix
med.corr()*100

,age,bmi,children,charges
age,100.000000,10.927188,4.246900,29.900819
bmi,10.927188,100.000000,1.275890,19.834097
children,4.246900,1.275890,100.000000,6.799823
charges,29.900819,19.834097,6.799823,100.000000


In [123]:
# Let's look at age vs. charges
px.scatter(med, x = "age", y = "charges", color = "smoker",)

It looks like there is a clear separation between smokers and non-smokers. It would be a good idea to split the dataset on that to have a more accurate model for one or the other.

In [124]:
# Get only the non-smokers
non_smoker = med[med['smoker']=="no"]

# Go ahead and drop the smoker column (redudancy)
non_smoker = non_smoker.drop("smoker", axis=1)
non_smoker.head()

,age,sex,bmi,children,charges
1,18,male,33.770,1,1725.55230
2,28,male,33.000,3,4449.46200
3,33,male,22.705,0,21984.47061
4,32,male,28.880,0,3866.85520
5,31,female,25.740,0,3756.62160


There's some categorical data in there. Let's change it to numerical with the `pd.get_dummies` function

In [125]:
# Change the categorical data to numerical
num_ns = pd.get_dummies(non_smoker)
num_ns.head()

,age,bmi,children,charges,sex_female,sex_male
1,18,33.770,1,1725.55230,0,1
2,28,33.000,3,4449.46200,0,1
3,33,22.705,0,21984.47061,0,1
4,32,28.880,0,3866.85520,0,1
5,31,25.740,0,3756.62160,1,0


Now, let's go ahead and put this into a model, starting with only looking at one variable: age. The code here should look quite familiar

In [126]:
# Divide into X and y
X = num_ns[['age']]
y = num_ns['charges']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# Create and train the model
reg_ns = LinearRegression()
reg_ns.fit(X_train, y_train)


LinearRegression()

In [127]:
# Generate Predictions
predicted = reg_ns.predict(X_test)
actual = np.array(y_test)

# Get MSE
mse_ns_age = mean_squared_error(y_pred=predicted, y_true=actual)

# Get RMSE
rmse_ns_age = mean_squared_error(y_pred=predicted, y_true=actual, squared=False)


MSE NS: 19603075.14431584
RMSE NS: 4427.536012763288


In [128]:
# Get only the smokers
smoker = med[med['smoker']=="yes"]

# Go ahead and drop the smoker column (redudancy)
smoker = smoker.drop("smoker", axis=1)
smoker.head()

# Change the categorical data to numerical
num_s = pd.get_dummies(smoker)
num_s.head()

# Divide into X and y
X = num_s[['age']]
y = num_s['charges']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# Create and train the model
reg_s = LinearRegression()
reg_s.fit(X_train, y_train)

# Generate Predictions
predicted = reg_s.predict(X_test)
actual = np.array(y_test)

# Get MSE
mse_s_age = mean_squared_error(y_pred=predicted, y_true=actual)

# Get RMSE
rmse_s_age = mean_squared_error(y_pred=predicted, y_true=actual, squared=False)


MSE S: 140278327.00197324
RMSE S: 11843.915188904944


## Let's visualize the trends

In [138]:
# Getting coefficients and intercept of the non smokers
coef_ns = reg_ns.coef_
intercept_ns = reg_ns.intercept_

# Creating the line
x_0_ns = 18
y_0_ns = coef_ns[0]*x_0 + intercept_ns
x_1_ns = 64
y_1_ns = coef_ns[0]*x_1 + intercept_ns

# Getting coefficients and intercept of the non smokers
coef_s = reg_s.coef_
intercept_s = reg_s.intercept_

# Creating the line
x_0_s = 18
y_0_s = coef_s[0]*x_0 + intercept_s
x_1_s = 64
y_1_s = coef_s[0]*x_1 + intercept_s

# Graphs the data and the line on plotly
fig = px.scatter(med, x="age", y="charges", color = 'smoker')
fig.add_shape(type='line', xref="x", yref="y",
    x0 = x_0_ns, y0 = y_0_ns, x1 = x_1_ns, y1 = y_1_ns,
    line = dict(
        color = "purple",
        width = 4,
    )   
)
fig.add_shape(type='line', xref="x", yref="y",
    x0 = x_0_s, y0 = y_0_s, x1 = x_1_s, y1 = y_1_s,
    line = dict(
        color = "forestgreen",
        width = 4,
    )   
)
fig.show()

In [139]:
# Metrics!
print("MSE S:",mse_s_age)
print("RMSE S:",rmse_s_age)
print("MSE NS:",mse_ns_age)
print("RMSE NS:",rmse_ns_age)

MSE S: 140278327.00197324
RMSE S: 11843.915188904944
MSE NS: 19603075.14431584
RMSE NS: 4427.536012763288


# Using Multiple Features for Linear Regression

What if we wanted to look at ALL of the different columns within the dataset (age, bmi, children, sex)?

We just add coefficients!

$$ charges = \beta_{1} * age + \beta_{2} * bmi + \beta_{3} * children + \beta_{4} * sex\_male + \beta_{5} * sex\_female + \beta_{0}$$

In [130]:
# Divide into X and y
X = num_ns.drop('charges', axis=1)
y = num_ns['charges']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# Create and train the model
reg_mult = LinearRegression()
reg_mult.fit(X_train, y_train)

LinearRegression()

In [131]:
# Generate Predictions
predicted = reg_mult.predict(X_test)
actual = np.array(y_test)

# Get MSE for both
mse_mult = mean_squared_error(y_pred=predicted, y_true=actual)
print("MSE for model with just age:", mse_age)
print("MSE for model with all features:", mse_mult)

# Get RMSE for both
rmse_mult = mean_squared_error(y_pred=predicted, y_true=actual, squared=False)
print("MSE for model with just age:", rmse_age)
print("MSE for model with all features:", rmse_mult)

MSE for model with just age: 19603075.14431584
MSE for model with all features: 19233917.064102437
MSE for model with just age: 4427.536012763288
MSE for model with all features: 4385.648990070049


It's going to be a little hard to graph this... Why do you think that is?